# Model building Pipeline using easyflow feature_encoders module

This module is a fusion between keras layers and tensorflow feature columns

In [1]:
import pandas as pd
import tensorflow as tf

In [2]:
# local imports
from easyflow.data.mapper import TensorflowDataMapper
from easyflow.feature_encoders.transformer import FeatureColumnTransformer, FeatureUnionTransformer
from easyflow.feature_encoders.feature_encoder import NumericalFeatureEncoder, EmbeddingFeatureEncoder, CategoricalFeatureEncoder

In [3]:
CSV_HEADER = [
    "age",
    "workclass",
    "fnlwgt",
    "education",
    "education_num",
    "marital_status",
    "occupation",
    "relationship",
    "race",
    "gender",
    "capital_gain",
    "capital_loss",
    "hours_per_week",
    "native_country",
    "income_bracket",
]

data_url = (
    "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
)
data_frame = pd.read_csv(data_url, header=None, names=CSV_HEADER)
labels = data_frame.pop("income_bracket")
labels_binary = 1.0 * (labels == " >50K")

print(f"Train dataset shape: {data_frame.shape}")

Train dataset shape: (32561, 14)


In [4]:
batch_size = 256
dataset_mapper = TensorflowDataMapper() 
dataset = dataset_mapper.map(data_frame, labels_binary)

train_data_set, val_data_set = dataset_mapper.split_data_set(dataset)
train_data_set = train_data_set.batch(batch_size)
val_data_set = val_data_set.batch(batch_size)

In [5]:
NUMERIC_FEATURE_NAMES = [
    "age",
    "education_num",
    "capital_gain",
    "capital_loss",
    "hours_per_week",
]

CATEGORICAL_FEATURES_NAMES = [
    "workclass",
    "marital_status",
    "relationship",
    "race",
    "gender"]

EMBEDDING_FEATURES_NAMES = ['education',
                            'occupation',
                            'native_country']

In [6]:
feature_encoder_list = [('numerical_features', NumericalFeatureEncoder(), NUMERIC_FEATURE_NAMES),
                        ('categorical_features', CategoricalFeatureEncoder(), CATEGORICAL_FEATURES_NAMES),
                        ('embedding_features_deep', EmbeddingFeatureEncoder(), EMBEDDING_FEATURES_NAMES),
                        ('embedding_features_wide', CategoricalFeatureEncoder(), EMBEDDING_FEATURES_NAMES)]

## Setting up feature layer and feature encoders

There are two main column transformer classes namely FeatureColumnTransformer and FeatureUnionTransformer. For this example we are going to build a Wide and Deep model architecture. So we will be using the FeatureColumnTransformer since it gives us more flexibility.

In [7]:
feature_layer_inputs, feature_encoders =  FeatureColumnTransformer(feature_encoder_list).transform(data_frame)

In [8]:
deep_features = feature_encoders['numerical_features']+\
                feature_encoders['categorical_features']+\
                feature_encoders['embedding_features_deep']

wide_features = feature_encoders['embedding_features_wide']


In [9]:
deep = tf.keras.layers.DenseFeatures(deep_features)(feature_layer_inputs)
deep = tf.keras.layers.BatchNormalization()(deep)

wide = tf.keras.layers.DenseFeatures(wide_features)(feature_layer_inputs)

for nodes in [128, 64, 32]:
    deep = tf.keras.layers.Dense(nodes, activation='relu')(deep)
    deep = tf.keras.layers.Dropout(0.5)(deep)

wide_and_deep = tf.keras.layers.concatenate([deep, wide])
output = tf.keras.layers.Dense(1, activation='sigmoid')(wide_and_deep)
model = tf.keras.Model(inputs=[v for v in feature_layer_inputs.values()], outputs=output)
model.compile(loss=tf.keras.losses.BinaryCrossentropy(label_smoothing=0.0),
              optimizer=tf.keras.optimizers.Adam(lr=0.001),
              metrics=[tf.keras.metrics.BinaryAccuracy(name='accuracy'), tf.keras.metrics.AUC(name='auc')])

In [10]:
model.fit(train_data_set, validation_data=val_data_set, epochs=10)

Epoch 1/10


/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/functional.py:543: UserWarning: Input dict contained keys ['fnlwgt'] which did not match any model input. They will be ignored by the model.
  [n for n in tensors.keys() if n not in ref_input_names])


96/96 [==============================] - 2s 16ms/step - loss: 0.4937 - accuracy: 0.7495 - auc: 0.7289 - val_loss: 0.5570 - val_accuracy: 0.6604 - val_auc: 0.8338
Epoch 2/10
96/96 [==============================] - 1s 10ms/step - loss: 0.3894 - accuracy: 0.8161 - auc: 0.8585 - val_loss: 0.4023 - val_accuracy: 0.7849 - val_auc: 0.8948
Epoch 3/10
96/96 [==============================] - 1s 10ms/step - loss: 0.3665 - accuracy: 0.8296 - auc: 0.8764 - val_loss: 0.3414 - val_accuracy: 0.8401 - val_auc: 0.9052
Epoch 4/10
96/96 [==============================] - 1s 10ms/step - loss: 0.3534 - accuracy: 0.8419 - auc: 0.8855 - val_loss: 0.3259 - val_accuracy: 0.8493 - val_auc: 0.9079
Epoch 5/10
96/96 [==============================] - 1s 10ms/step - loss: 0.3486 - accuracy: 0.8422 - auc: 0.8894 - val_loss: 0.3234 - val_accuracy: 0.8484 - val_auc: 0.9093
Epoch 6/10
96/96 [==============================] - 1s 10ms/step - loss: 0.3438 - accuracy: 0.8434 - auc: 0.8925 - val_loss: 0.3190 - val_accuracy